In [3]:
import requests
import os
import gzip
import shutil
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np


In [4]:

calendar = pd.read_csv('calendar.csv')
listings = pd.read_csv('listings.csv')
reviews = pd.read_csv('reviews.csv')
nneighbourhoods = pd.read_csv('neighbourhoods.csv')


C:\Users\estev\AppData\Local\Temp\ipykernel_8956\2912616903.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  calendar = pd.read_csv('calendar.csv')


In [17]:
columnas50 = listings.columns[listings.isnull().sum() / listings.shape[0] * 100 > 30]
for i in columnas50:
    if i in listings.columns:
        listings.drop(columns = i, inplace=True)

listings.head(1)

,id,listing_url,scrape_id,last_scraped,source,name,description,picture_url,host_id,host_url,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,7381,https://www.airbnb.com/rooms/7381,20240825194210,2024-08-26,city scrape,Chalet Chocolat in medieval alpine village,Beautifully renovated two-room apartment in ch...,https://a0.muscache.com/pictures/23261/dd61f58...,19480,https://www.airbnb.com/users/show/19480,...,4.7,4.8,4.75,4.67,f,1,1,0,0,0.42


In [27]:
data = listings
data.iloc[:1,:9] 

,id,listing_url,scrape_id,last_scraped,source,name,description,picture_url,host_id
0,7381,https://www.airbnb.com/rooms/7381,20240825194210,2024-08-26,city scrape,Chalet Chocolat in medieval alpine village,Beautifully renovated two-room apartment in ch...,https://a0.muscache.com/pictures/23261/dd61f58...,19480


Vamos a describir brevemente las columnas del dataset 'listings'. Comenzaremos desde la columna 1 hasta la columna 9.

1. **id**: 
    - ID del anuncio.

2. **listing_url**: 
    - URL del anuncio.

3. **scrape_id**: 
    - Identificador del proceso de *scraping*. 

4. **last_scraped**: 
    - Fecha en la que se realizó el último scrape de este anuncio.

5. **source**: 
    - Fuente del scrape.

6. **name**: 
    - Nombre del anuncio.

7. **description**: 
    - Descripción de la casa del anuncio, que suele ser hecha por el propietario o por una agencia inmobiliaria.

8. **picture_url**: 
    - Url de la imagen principal del anuncio.

9. **host_id**: 
    - Id del propietario.


In [28]:
data.iloc[:1,9:19] 

,host_url,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url
0,https://www.airbnb.com/users/show/19480,Paula,2009-05-31,"Geneva, Switzerland",within an hour,100%,67%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...


Continuamos desde la columna 10 hasta la 19.

10. **host_url**: 
    - Url del propietario (dentro del entorno de airbnb).

11. **host_name**:
    - Nombre del propietario.

12. **host_since**:
    - Fecha de adquisición de la casa por parte del propietario.

13. **host_location**: 
    - Localidad del propietario.

14. **host_response_time**: 
    - Tiempo que tarda en responder el propietario a una petición.

15. **host_response_rate**: 
    - Ratio de las veces que responde el propietario a una solicitud.

16. **host_acceptance_rate**: 
    - Ratio de las veces que el propietario acepta una solicitud.

17. **host_is_superhost**: 
    - Booleana que indica si el propietario está considerado un superhost (super-anfitrión). 

18. **host_thumbnail_url**: 
    - Url de la foto de perfil del anfitrión.

19. **host_picture_url**: 
    - URL de la foto de perfil del anfitrión.



In [29]:
data.iloc[:1,19:29] 

,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type
0,1,3,"['email', 'phone']",t,t,Ormont-Dessus,Aigle,46.35447,7.13187,Entire rental unit


Continuamos desde la columna 20 hasta la 29.

20. **host_listings_count**:
    - Número de anuncios activos, en el momento de la consulta, que tiene el anfitrión.

21. **host_total_listings_count**: 
    - Número total de anuncios que ha publicado el anfitrión en la página.
    
22. **host_verifications**: 
    - Métodos de verificación del anfitrión (email, teléfono, etc.).

23. **host_has_profile_pic**: 
    - Booleana que indica si el anfitrión tiene foto de perfil.

24. **host_identity_verified**: 
    - Booleana que indica si la identidad del anfitrión ha sido verificada.

25. **neighbourhood_cleansed**: 
    - Nombre del barrio donde se encuentra la propiedad.

26. **neighbourhood_group_cleansed**: 
    - Nombre del grupo de barrios donde se encuentra la propiedad.

27. **latitude**: 
    - Latitud de la propiedad.

28. **longitude**: 
    - Longitud de la propiedad.

29. **property_type**: 
    - Tipo de propiedad (apartamento, casa, etc.).



In [30]:
data.iloc[:1,29:39] 

,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights
0,Entire home/apt,3,1.0,1 bath,1.0,2.0,"[""Private backyard \u2013 Not fully fenced"", ""...",$87.00,3,180


Continuamos desde la columna 30 hasta la 39.

30. **room_type**: 
    - Tipo de habitación

31. **accommodates**: 
    - Número de personas que puede alojar la propiedad.

32. **bathrooms**: 
    - Número de baños en la propiedad.

33. **bathrooms_text**: 
    - Descripción textual del número de baños.

34. **bedrooms**: 
    - Número de habitaciones en la propiedad.

35. **beds**: 
    - Número de camas disponibles en la propiedad.

36. **amenities**: 
    - Lista de comodidades disponibles en la propiedad.

37. **price**: 
    - Precio por noche de la propiedad.

38. **minimum_nights**: 
    - Número mínimo de noches que se puede reservar la propiedad.

39. **maximum_nights**: 
    - Número máximo de noches que se puede reservar la propiedad.





In [39]:
data.iloc[:1,39:50] 

,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365
0,3,3,180,180,3.0,180.0,t,30,60,90,358


Continuamos desde la columna 40 hasta la 49.

40. **minimum_minimum_nights**: 
    - Número mínimo de noches que se puede reservar la propiedad en cualquier momento.

41. **maximum_minimum_nights**: 
    - Número máximo de noches mínimas que se puede reservar la propiedad.

42. **minimum_maximum_nights**: 
    - Número mínimo de noches máximas que se puede reservar la propiedad.

43. **maximum_maximum_nights**: 
    - Número máximo de noches máximas que se puede reservar la propiedad.

44. **minimum_nights_avg_ntm**: 
    - Promedio de noches mínimas que se puede reservar la propiedad en el próximo mes.

45. **maximum_nights_avg_ntm**: 
    - Promedio de noches máximas que se puede reservar la propiedad en el próximo mes.

46. **has_availability**: 
    - Indica si la propiedad tiene disponibilidad.

47. **availability_30**: 
    - Número de noches disponibles en los próximos 30 días.

48. **availability_60**: 
    - Número de noches disponibles en los próximos 60 días.

49. **availability_90**: 
    - Número de noches disponibles en los próximos 90 días.

50. **availability_365**: 
    - Número de noches disponibles en los próximos 365 días.


In [40]:
data.iloc[:1,50:60] 

,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin
0,2024-08-26,62,4,0,2012-07-31,2024-07-09,4.75,4.71,4.85,4.7


Continuamos desde la columna 50 hasta la 59.

51. **calendar_last_scraped**: Fecha en la que se realizó el último scrape del calendario.

52. **number_of_reviews**: Número total de reseñas que ha recibido la propiedad.

53. **number_of_reviews_ltm**: Número de reseñas recibidas en los últimos 12 meses.

54. **number_of_reviews_l30d**: Número de reseñas recibidas en los últimos 30 días.

55. **first_review**: Fecha de la primera reseña recibida.

56. **last_review**: Fecha de la última reseña recibida.

57. **review_scores_rating**: Puntuación general de la propiedad basada en las reseñas.

58. **review_scores_accuracy**: Puntuación de la precisión de la descripción de la propiedad.

59. **review_scores_cleanliness**: Puntuación de la limpieza de la propiedad.

60. **review_scores_checkin**: Puntuación del proceso de check-in.



In [41]:
data.iloc[:1,60:70] 

,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,4.8,4.75,4.67,f,1,1,0,0,0.42


Continuamos desde la columna 60 hasta la 69.

61. **review_scores_communication**: Puntuación de la comunicación con el anfitrión.

62. **review_scores_location**: Puntuación de la ubicación de la propiedad.

63. **review_scores_value**: Puntuación de la relación calidad-precio de la propiedad.

64. **instant_bookable**: Indica si la propiedad se puede reservar instantáneamente.

65. **calculated_host_listings_count**: Número total de anuncios del anfitrión.

66. **calculated_host_listings_count_entire_homes**: Número de anuncios de casas completas del anfitrión.

67. **calculated_host_listings_count_private_rooms**: Número de anuncios de habitaciones privadas del anfitrión.

68. **calculated_host_listings_count_shared_rooms**: Número de anuncios de habitaciones compartidas del anfitrión.

69. **reviews_per_month**: Número de reseñas por mes.

